In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import KFold

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
matches = pd.DataFrame.from_csv('data/match_history.csv',index_col=None, encoding='ISO-8859-1')
bracket = pd.DataFrame.from_csv('data/us_open_bracket.csv', index_col=None, encoding='ISO-8859-1')
ranking = pd.DataFrame.from_csv('data/atp_rankings.csv', index_col=0, encoding='ISO-8859-1')

In [7]:
hard_court = matches[(matches.surface=='Hard')]
rankings = hard_court.iloc[:,15:17]

In [6]:
new_df = pd.DataFrame()
new_df['highest_rank'] = rankings.min(axis=1)
new_df['rank_diff'] = rankings.p1_rank - rankings.p2_rank
new_df['winner'] = hard_court.Winner

#new_df = pd.DataFrame( { "winner":hard_court.Winner, "rank_diff":hard_court.p1_rank - hard_court.p2_rank} )
new_df.head()

,highest_rank,rank_diff,winner
0,63,-14,1
1,5,-51,1
2,40,-615,1
3,65,22,2
4,81,117,2


In [15]:
# Split the data into a training and test set.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(new_df[['highest_rank','rank_diff']].values, 
                                              (new_df.winner == 1).values)

clf = LogisticRegression()
# Fit the model on the trainng data.
clf.fit(Xtrain, Ytrain)
# Print the accuracy from the testing data.
print(accuracy_score(clf.predict(Xtest), Ytest))

0.660913872452


In [17]:
def cv_score(clf, x, y, score_func=accuracy_score):
    result = 0
    nfold = 5
    for train, test in KFold(nfold).split(x): # split data into train/test groups, 5 times
        clf.fit(x[train], y[train]) # fit
        result += score_func(clf.predict(x[test]), y[test]) # evaluate score function on held-out data
    return result / nfold # average

print(cv_score(clf, Xtrain, Ytrain))

0.664273972603


In [27]:

# Use grid search in cross validation and training of the penalty (regularization) parameter, C
param_grid = [  {'C': [ 10, 100], 'kernel': ['rbf']}]
clf = sklearn.model_selection.GridSearchCV(sklearn.svm.SVC(kernel='rbf', C=1), param_grid)
clf.fit(Xtrain, Ytrain)
clf.score(Xtest, Ytest)
    


KeyboardInterrupt: 